In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
sfdata = pd.read_csv('train.csv', parse_dates=['Dates'])
sfdata.head(5)
sfdata['Year'] = sfdata['Dates'].map(lambda x:x.year)
sfdata['Month'] = sfdata['Dates'].map(lambda x: x.month)
sfdata['Time'] = sfdata['Dates'].map(lambda x: x.time)

In [3]:
sfdata.head(5)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Year,Month,Time
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,<built-in method time of Timestamp object at 0...
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,<built-in method time of Timestamp object at 0...
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,<built-in method time of Timestamp object at 0...
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,<built-in method time of Timestamp object at 0...
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,<built-in method time of Timestamp object at 0...


In [4]:
def day_map(x):
    '''x is a time stamp object'''
    time = x()
    hour = time.hour
    if(hour >= 6 and hour < 12): return 1
    elif(hour >= 12 and hour < 17): return 2
    elif(hour >= 17 and hour < 21): return 3
    else: return 4

In [5]:
sfdata['Hour'] = sfdata['Time'].map(day_map)

In [6]:
sfdata.head(5)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Year,Month,Time,Hour
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,<built-in method time of Timestamp object at 0...,4
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,<built-in method time of Timestamp object at 0...,4
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,<built-in method time of Timestamp object at 0...,4
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,<built-in method time of Timestamp object at 0...,4
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,<built-in method time of Timestamp object at 0...,4


In [7]:
sfdata['PdDistrict'].unique()
#len(sfdata['Category'].unique())

array(['NORTHERN', 'PARK', 'INGLESIDE', 'BAYVIEW', 'RICHMOND', 'CENTRAL',
       'TARAVAL', 'TENDERLOIN', 'MISSION', 'SOUTHERN'], dtype=object)

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit_transform(sfdata['PdDistrict'])

array([4, 4, 4, ..., 7, 7, 0])

In [9]:
sfdata_enc = pd.DataFrame()

In [10]:
sfdata_enc['Category'] = le.fit_transform(sfdata['Category'])
sfdata_enc['DayOfWeek'] = le.fit_transform(sfdata['DayOfWeek'])
sfdata_enc['PdDistrict'] = le.fit_transform(sfdata['PdDistrict'])
#sfdata_enc['X'] = sfdata['X'].copy()
#sfdata_enc['Y'] = sfdata['Y'].copy()
sfdata_enc['TimeofDay'] = le.fit_transform(sfdata['Hour'])

#sfdata_enc.to_csv('sfd_0.csv',sep=',', na_rep='', header=False, index=False)
sfdata_enc.head(5)


,Category,DayOfWeek,PdDistrict,TimeofDay
0,37,6,4,3
1,21,6,4,3
2,21,6,4,3
3,16,6,4,3
4,16,6,5,3


In [11]:
enc = preprocessing.OneHotEncoder(sparse=False)
sfdata_enc_one_hot_matrix = enc.fit_transform(sfdata_enc[['Category', 'DayOfWeek', 'TimeofDay']])
#sfdata_enc_one_hot_matrix = sfdata_enc_one_hot_matrix.astype(np.uint8)
district_matrix = sfdata_enc['PdDistrict'].as_matrix()

district_matrix = district_matrix.reshape((len(district_matrix), 1))



In [12]:
sfdata_enc_one_hot_matrix = np.hstack((district_matrix, sfdata_enc_one_hot_matrix))
sfdata_enc_one_hot = pd.DataFrame(sfdata_enc_one_hot_matrix)
sfdata_enc_one_hot.to_csv('sfd_train_one_hot_2.csv', sep=',', na_rep='',header=False, index=False)

In [13]:
sfdata_enc_one_hot.head(5)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [40]:
sfdata_enc['Category'].unique()

array([37, 21, 16, 36, 35, 20, 25,  1, 38,  4, 32,  8, 12,  7, 30, 27, 34,
       19, 13, 15, 26,  6, 28, 23,  5,  0, 11, 17,  3,  9, 31, 18, 29, 10,
       14,  2, 33, 24, 22])

In [37]:
# For one hot encoding of data frame
enc = preprocessing.OneHotEncoder(sparse=False)
sfdata_enc_one_hot_matrix = enc.fit_transform(sfdata_enc[['DayOfWeek', 'PdDistrict', 'TimeofDay']])
sfdata_enc_one_hot_matrix = sfdata_enc_one_hot_matrix.astype(np.uint8)
category_matrix = sfdata_enc['Category'].as_matrix()

category_matrix = category_matrix.reshape((len(category_matrix), 1))

sfdata_enc_one_hot_matrix = np.hstack((category_matrix, sfdata_enc_one_hot_matrix))
sfdata_enc_one_hot = pd.DataFrame(sfdata_enc_one_hot_matrix)
sfdata_enc_one_hot.to_csv('sfd_train_one_hot.csv', sep=',', na_rep='',header=False, index=False)

In [38]:
sfdata_enc_one_hot.head(5)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,37,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
1,21,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
2,21,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
3,16,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
4,16,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1


In [ ]:
sfdata_enc.shape
sfdata_enc.head(10)

In [9]:
sfdata_enc_mat = sfdata_enc.as_matrix()

In [ ]:
sfdata_enc_mat.shape
sfdata_enc_mat[:5]

In [13]:
X = sfdata_enc_mat[:,1:]
y = np.asarray(sfdata_enc_mat[:,0], dtype=np.uint8)

#enc_x = preprocessing.OneHotEncoder(sparse=False)
#X_enc = enc_x.fit_transform(X)

#num_classes = len(sfdata['Category'].unique())
#y_enc = np.eye(num_classes)[y]

#X_enc[:,21] -= np.mean(X_enc[:,21], axis=0)
#X_enc[:,21] /= np.std(X_enc[:,21], axis=0)

#X_enc[:,22] -= np.mean(X_enc[:,22], axis=0)
#X_enc[:,22] /= np.std(X_enc[:,22], axis=0)
print X.shape
print y.shape




(878049, 3)
(878049,)


In [14]:
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

X_train, X_val, y_train, y_val = cross_validation.train_test_split(X, y, test_size=0.20, random_state=42)
print 'Training Shape: ', X_train.shape
print 'Validation Shape: ', X_val.shape, '\n'

Training Shape:  (702439, 3)
Validation Shape:  (175610, 3) 



In [ ]:
clf = svm.LinearSVC(multi_class='ovr')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)

print 'Accuracy is: ', 100 * accuracy_score(y_val, y_pred)